In [ ]:
!pip install awscli
import awscli
from google.colab import drive
!pip install pydicom
drive.mount("/content/drive", force_remount=True)

In [ ]:
!cat /content/drive/My\ Drive/config/awscli.ini
path = "/content/drive/My Drive/config/awscli.ini"

import os
!export AWS_SHARED_CREDENTIALS_FILE=/content/drive/My\ Drive/config/awscli.ini
path = "/content/drive/My Drive/config/awscli.ini"
os.environ['AWS_SHARED_CREDENTIALS_FILE'] = path

!aws s3 cp s3://medical-image-segmentation/lungs/70-10-20-3D/train.zip .
!aws s3 cp s3://medical-image-segmentation/lungs/70-10-20-3D/val.zip .
!aws s3 cp s3://medical-image-segmentation/lungs/70-10-20-3D/test.zip .
!aws s3 cp s3://medical-image-segmentation/lungs/70-10-20-3D/train-output.zip .
!aws s3 cp s3://medical-image-segmentation/lungs/70-10-20-3D/val-output.zip .
!aws s3 cp s3://medical-image-segmentation/lungs/70-10-20-3D/test-output.zip .

!unzip train.zip
!unzip val.zip
!unzip test.zip
!unzip train-output.zip
!unzip val-output.zip
!unzip test-output.zip

In [ ]:
# coding: utf-8
import os
import tensorflow as tf
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow import keras
!pip install keras_unet
from keras_unet.models import custom_unet

SEED = 909
BATCH_SIZE_TRAIN = 4
BATCH_SIZE_VAL = 4
BATCH_SIZE_TEST = 4
IMAGE_HEIGHT=512
IMAGE_WIDTH=512
IMG_SIZE = (IMAGE_HEIGHT,IMAGE_WIDTH)
NUM_TRAIN = 6651
NUM_VAL = 932
NUM_TEST = 1950

def create_train(img_path, mask_path, BATCH_SIZE):
    data_gen_args = dict(rescale=1./255)
    img_datagen = ImageDataGenerator(**data_gen_args)
    mask_datagen = ImageDataGenerator(**data_gen_args)
    
    img_generator = img_datagen.flow_from_directory(img_path, target_size=IMG_SIZE, class_mode=None,
       color_mode='grayscale', batch_size=BATCH_SIZE, seed=SEED)

    mask_generator = mask_datagen.flow_from_directory(mask_path, target_size=IMG_SIZE, class_mode=None,
       color_mode='grayscale', batch_size=BATCH_SIZE, seed=SEED)
    return zip(img_generator, mask_generator)

train_img_path = os.path.join('train-output', 'images')
train_mask_path = os.path.join('train-output', 'masks')

val_img_path = os.path.join('val-output', 'images')
val_mask_path = os.path.join('val-output', 'masks')

test_img_path = os.path.join('test-output', 'images')
test_mask_path = os.path.join('test-output', 'masks')

train_generator = create_train(train_img_path, train_mask_path, BATCH_SIZE_TRAIN)
val_generator = create_train(val_img_path, val_mask_path, BATCH_SIZE_TRAIN)
test_generator = create_train(test_img_path, test_mask_path, BATCH_SIZE_TRAIN)

NUM_OF_EPOCHS = 100

In [ ]:
import matplotlib.pyplot as plt

def display(display_list):
    plt.figure(figsize=(15,15))
    title = ['Input', 'True Mask', 'Predicted Mask']
    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]), cmap='gray')
    plt.show()

def show_prediction(datagen, num=1):
    for i in range(0,num):
        image,mask = next(datagen)
        pred_mask = model.predict(image)[0] > 0.5
        display([image[0], mask[0], pred_mask])

In [ ]:
img_arr = [array[i] for i in range(140)]
for img in img_arr:
  plt.imshow(img, cmap="gray")
  plt.show()

In [ ]:
show_prediction(test_generator, 10)

In [ ]:
import keras.backend as K
import math
def DiceLoss(targets, inputs, smooth=1e-6):
    inputs = K.flatten(inputs)
    targets = K.flatten(targets)
    
    intersection = K.sum(targets * inputs)
    dice = (2*intersection + smooth) / (K.sum(targets) + K.sum(inputs) + smooth)
    return 1 - dice

In [ ]:
model = custom_unet(
    input_shape=(512, 512, 1),
    use_batch_norm=True,
    num_classes=1,
    filters=64,
    dropout=0.25,
    output_activation='sigmoid')

In [ ]:
EPOCH_STEP_TRAIN = NUM_TRAIN // BATCH_SIZE_TRAIN
EPOCH_STEP_VAL = NUM_VAL // BATCH_SIZE_VAL
EPOCH_STEP_TEST = NUM_TEST // BATCH_SIZE_TEST
model.compile(optimizer='adam', loss=DiceLoss, metrics=[DiceLoss, tf.keras.metrics.Precision(), tf.keras.metrics.Recall()], run_eagerly=True)

In [ ]:
model.fit_generator(generator=train_generator, 
                    steps_per_epoch=EPOCH_STEP_TRAIN, 
                    validation_data=val_generator, 
                    validation_steps=EPOCH_STEP_VAL,
                    epochs=30)

In [ ]:
model.save(f'drive/MyDrive/UNET6-A99-P93-R88-{IMAGE_HEIGHT}_{IMAGE_WIDTH}.h5')

In [ ]:
model = keras.models.load_model(f'drive/MyDrive/UNET6-A99-P93-R88-{IMAGE_HEIGHT}_{IMAGE_WIDTH}.h5', custom_objects={"DiceLoss": DiceLoss})

# 2D Surface Dice Evaluation

In [ ]:
def dice_gen(img_path, mask_path, BATCH_SIZE):
    data_gen_args = dict(rescale=1./255)
    img_datagen = ImageDataGenerator(**data_gen_args)
    mask_datagen = ImageDataGenerator(**data_gen_args)
    
    img_generator = img_datagen.flow_from_directory(img_path, target_size=IMG_SIZE, class_mode=None,
       color_mode='grayscale', batch_size=BATCH_SIZE, seed=SEED, shuffle=False)

    mask_generator = mask_datagen.flow_from_directory(mask_path, target_size=IMG_SIZE, class_mode=None,
       color_mode='grayscale', batch_size=BATCH_SIZE, seed=SEED, shuffle=False)
    return img_generator, mask_generator

In [ ]:
from pydicom import dcmread
import statistics
from os import listdir
from os.path import join
import matplotlib.pyplot as plt

!pip install ctg-surface-distance
from ctg-surface-distance import compute_surface_distances, compute_surface_dice_at_tolerance

def get_distances(path):
  distances = {}
  for subdir in listdir(path):
    for image in listdir(join(path, subdir, "images")):
      dimensions = dcmread(join(path, subdir, "images", image)).PixelSpacing
      png_name = "lung_l/" + subdir + "-" + image + ".png"
      distances[png_name] = [d for d in dimensions]                             # change this ratio if image dimensions change
  return distances

def get_surface_dice_values(img_path, mask_path, batch_size, iterations, pixel_distances):
  index = 0
  dice_list = []
  image_gen, mask_gen = dice_gen(img_path, mask_path, batch_size)
  files = image_gen.filenames

  for i in range(iterations):
    image, mask = next(image_gen), next(mask_gen)
    pred_masks = model.predict(image)
  
    for j in range(batch_size):
      pred_mask = pred_masks[j] > 0.5                                           # not sure if this is correct thing to do
      bool_mask = mask[j].astype(bool)
      mask_gt = bool_mask.reshape((512, 512))
      predicted_mask = pred_mask.reshape((512, 512))

      surface_distances = compute_surface_distances(mask_gt, predicted_mask, pixel_distances[files[index]])
      surface_dice = compute_surface_dice_at_tolerance(surface_distances, 1.9)
      
      dice_list.append(surface_dice)
      index += 1
  return dice_list
  
def compute_surface_dice_stats(dice_list):
  filtered_dice_list = []
  plt.hist(filtered_dice_list, bins=20)
  plt.show()
  for val in dice_list:
    if not math.isnan(val):
      filtered_dice_list.append(val)
  filtered_dice_list.sort()
  mean = sum(filtered_dice_list) / len(filtered_dice_list)
  median = filtered_dice_list[len(filtered_dice_list) // 2]
  std_dev = statistics.stdev(filtered_dice_list)
  return mean, median, std_dev, filtered_dice_list


In [ ]:
# split can be either train, val or test

def surface_dice(split):
  if split == "train":
    img_path, mask_path = train_img_path, train_mask_path
    batch_size, iterations = BATCH_SIZE_TRAIN, EPOCH_STEP_TRAIN
  elif split == "val":
    img_path, mask_path = val_img_path, val_mask_path
    batch_size, iterations = BATCH_SIZE_VAL, EPOCH_STEP_VAL
  elif split == "test":
    img_path, mask_path = test_img_path, test_mask_path
    batch_size, iterations = BATCH_SIZE_TEST, EPOCH_STEP_TEST
  else:
    assert False, "invalid split"
  
  pixel_distances = get_distances(split)
  dice_list = get_surface_dice_values(img_path, mask_path, batch_size, iterations, pixel_distances)
  return compute_surface_dice_stats(dice_list)
  

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.dpi']= 500
plt.hist(filtered_list, bins=50)
plt.show()

# 3D Surface Dice Evaluation

In [ ]:
from pydicom import dcmread
import statistics
from os import listdir
from os.path import join
import matplotlib.pyplot as plt
import numpy as np

!pip install ctg-surface-distance
from ctg-surface-distance import compute_surface_distances, compute_surface_dice_at_tolerance

def get_3D_prediction(img_path, mask_path, model, dimensions):
  img_gen, mask_gen = dice_gen(img_path, mask_path, 1)
  num_images = len(img_gen.filenames)
  file_names = img_gen.filenames
  patient = ""
  dice_scores = []
  for i in range(num_images):
    curr_patient = file_names[i][7:19]
    if curr_patient != patient:
      if patient != "":
        # compute the 3d dice here
        surface_distances = compute_surface_distances(output_mask, output_pred, dimensions[patient])
        surface_dice = compute_surface_dice_at_tolerance(surface_distances, 1.9)
        dice_scores.append(surface_dice)
      
      image, mask = next(img_gen), next(mask_gen)
      output_pred = model.predict(image).reshape(1, 512, 512) > 0.5
      output_mask = mask.reshape(1, 512, 512).astype(bool)
      count = 1
      patient = curr_patient
    else:
      image, mask = next(img_gen), next(mask_gen)
      slice_pred = model.predict(image).reshape(1, 512, 512) > 0.5
      output_pred = np.vstack((output_pred, slice_pred))
      output_mask = np.vstack((output_mask, mask.reshape(1, 512, 512).astype(bool)))
      count += 1
  
  surface_distances = compute_surface_distances(output_mask, output_pred, dimensions[patient])
  surface_dice = compute_surface_dice_at_tolerance(surface_distances, 1.9)
  dice_scores.append(surface_dice)

  return dice_scores

def get_3D_dimensions(split):
  dimensions = {}
  for patient in listdir(split):
    img = listdir(join(split, patient, "images"))[0]
    image_data = dcmread(join(split, patient, "images", img))
    pixel_spacing = image_data.PixelSpacing
    thickness = image_data.SliceThickness
    dimensions[patient] = (thickness, pixel_spacing[0], pixel_spacing[1])
  return dimensions

In [ ]:
dimensions = get_3D_dimensions("test")
dice_scores = get_3D_prediction(test_img_path, test_mask_path, model, dimensions)

Found 1604 images belonging to 1 classes.
Found 1604 images belonging to 1 classes.


In [ ]:
print(dimensions)
print(dice_scores)

{'LCTSC-S1-005': ("3.0", "0.9765625", "0.9765625"), 'LCTSC-S1-202': ("3.0", "0.9765625", "0.9765625"), 'LCTSC-S1-011': ("3.0", "0.9765625", "0.9765625"), 'LCTSC-S3-005': ("2.0", "1.171875", "1.171875"), 'LCTSC-S2-011': ("2.5", "0.976562", "0.976562"), 'LCTSC-S2-008': ("2.5", "0.976562", "0.976562"), 'LCTSC-S3-008': ("2.0", "0.976562", "0.976562"), 'LCTSC-S1-008': ("3.0", "0.9765625", "0.9765625"), 'LCTSC-S2-005': ("2.5", "0.976562", "0.976562"), 'LCTSC-S3-011': ("3.0", "1.171875", "1.171875")}
[0.9836000351507326, 0.962343696909276, 0.9372266811615954, 0.9709484560692622, 0.9478634316012897, 0.9282971252200365, 0.891586564570017, 0.8418964323295772, 0.9590294026342496, 0.7793676119737829]


# Code below for testing purposes

In [ ]:
index = 0
dice_list = []
for i in range(1662):
  image,mask = next(image_gen), next(mask_gen)
  pred_masks = model.predict(image)
  for j in range(4):
    pred_mask = pred_masks[j] > 0.5
    bool_mask = mask[j].astype(bool)
    
    mask_gt = bool_mask.reshape((512, 512))
    predicted_mask = pred_mask.reshape((512, 512))

    surface_distances = compute_surface_distances(mask_gt, predicted_mask, [1.9, 1.9])
    surface_dice = compute_surface_dice_at_tolerance(surface_distances, 1.9)
    display([image[j], mask[j], pred_mask])
    print("surface dice: " + str(surface_dice))
    dice_list.append(surface_dice)
    index += 1

In [ ]:
count = 0
total = 0
for val in dice_list:
  if not math.isnan(val):
    count += 1
    total += val

print(total/count)

0.8876650726948787


In [ ]:
from pydicom import dcmread

def get_distances(path):
  distances = {}
  for subdir in listdir(path):
    for image in listdir(join(path, subdir, "images")):
      dimensions = dcmread(join(path, subdir, "images", image)).PixelSpacing
      png_name = "lung_l/" + subdir + "-" + image + ".png"
      distances[png_name] = [d for d in dimensions]                         # change this ratio if image dimensions change
  return distances

distances = get_distances("val")

In [ ]:
image_gen, mask_gen = dice_gen(train_img_path, train_mask_path, BATCH_SIZE_TRAIN)
files = image_gen.filenames

Found 6651 images belonging to 1 classes.
Found 6651 images belonging to 1 classes.


In [ ]:
def 3D-surface_dice(split):
  dimensions = {}
  for patient in listdir(split):
    pixel_spacing = dcmread()